# JulES as a medium-term prognosis model

### Import packages

In [1]:
#Pkg.instantiate()
using Pkg; Pkg.status()
#Pkg.add("CSV"); Pkg.add("Revise"); Pkg.add("Plots"); Pkg.add("PlotlyJS"); Pkg.add("PrettyTables")
# Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# Pkg.develop(path=joinpath(dirname(dirname(pwd())),"TuLiPa")); Pkg.status() # go to development version
#Pkg.undo(); Pkg.status() # go back to main package version
#Pkg.add(url="https://github.com/NVE/TuLiPa.git", rev="redesign_JulES"); Pkg.status() # alternative go back to latest version

Project JulES v0.1.0
Status `X:\Prosjekter\2022_FoU_markedsmodell_julia\Brukere\mary\JulES\Project.toml`
  [336ed68f] CSV v0.10.14
  [aaaa29a8] Clustering v0.15.7
  [a93c6f00] DataFrames v1.6.1
  [aaf54ef3] DistributedArrays v0.6.7
⌃ [31c24e10] Distributions v0.25.107
  [7073ff75] IJulia v1.24.2
  [682c06a0] JSON v0.21.4
  [f0f68f2c] PlotlyJS v0.18.13
  [91a5bcdd] Plots v1.40.4
⌃ [aea7be01] PrecompileTools v1.2.0
  [08abe8d2] PrettyTables v2.3.2
  [295af30f] Revise v3.5.14
  [970f5c25] TuLiPa v0.1.0 `https://github.com/NVE/TuLiPa.git#redesign_JulES`
⌃ [ddb6d928] YAML v0.4.9
  [ade2ca70] Dates
  [8ba89e20] Distributed
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [2]:
using DataFrames, Statistics, JSON, Distributed, Clustering, YAML, Distributions, Revise, Plots, PrettyTables, Random
plotlyjs(); # uncomment for interactive plots

WebIO._IJuliaInit()

In [3]:
# config = YAML.load_file(joinpath("data", "config_jules_prognose.yml")) # config without datasets
config = YAML.load_file(joinpath("data", "config_jules_prognose_demo.yml")) # config with NVE datasets
weatheryear = config["main"]["weatheryears"][1]
datayear = config["main"]["datayears"][1]

2024

### Prepare parallell processing - import code on all cores

In [4]:
const numcores = config["main"]["numcores"]

if nprocs() < numcores
    addprocs(numcores - nprocs())
end

@show nprocs();

nprocs() = 10


In [5]:
@everywhere using TuLiPa, Dates
# @everywhere include(joinpath(dirname(dirname(pwd())),"TuLiPa/src/TuLiPa.jl"));

In [6]:
@everywhere using JulES

[ Info: Precompiling JulES [05c5cb9d-dcc3-436a-b9c7-df36424a75d6]


In [7]:
function getdataset(config, weatheryear)
    settings = config[config["main"]["settings"]]

    sti_dataset = joinpath(config["main"]["inputpath"], "static_input")
    weekstart = config["main"]["weekstart"]

    sti_dataset1 = joinpath(config["main"]["inputpath"], "Uke_$weekstart", "input")

    exd = JSON.parsefile(joinpath(sti_dataset1, "exogenprices_prognose1.json"))
    exogen = JulES.getelements(exd, sti_dataset1)

    add = JSON.parsefile(joinpath(sti_dataset, "aggdetd2.json"))
    aggdetd = JulES.getelements(add, sti_dataset)

    ipad = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoseragg$weatheryear.json"))
    agginflow = JulES.getelements(ipad, sti_dataset1)

    thd = JSON.parsefile(joinpath(sti_dataset, "termisk1.json"))
    thermal = JulES.getelements(thd, sti_dataset)

    wsd = JSON.parsefile(joinpath(sti_dataset, "vindsol.json"))
    windsol = JulES.getelements(wsd, sti_dataset)

    trd = JSON.parsefile(joinpath(sti_dataset1, "nett.json"))
    transm = JulES.getelements(trd)

    cod = JSON.parsefile(joinpath(sti_dataset, "forbruk5.json"))
    cons = JulES.getelements(cod, sti_dataset)

    fpd = JSON.parsefile(joinpath(sti_dataset1, "brenselspriser.json"))
    fuel = JulES.getelements(fpd, sti_dataset1)

    nud = JSON.parsefile(joinpath(sti_dataset1, "nuclear.json"))
    nuclear = JulES.getelements(nud, sti_dataset1)

    dse = JSON.parsefile(joinpath(sti_dataset, "tidsserier_detd.json"))
    detdseries = JulES.getelements(dse, sti_dataset)

    dda = JSON.parsefile(joinpath(sti_dataset, "dataset_detd.json"))
    detdstructure = JulES.getelements(dda)

    ipd = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoser$weatheryear.json"))
    inflow = JulES.getelements(ipd, sti_dataset1)

    progelements = vcat(exogen, aggdetd, thermal, windsol, transm, cons, agginflow, fuel, nuclear)
    aggstartmagdict = JSON.parsefile(joinpath(sti_dataset1, "aggstartmagdict.json"), dicttype=Dict{String, Float64})

    if JulES.get_onlyagghydro(settings)
        startmagdict = Dict()
        detailedrescopl = Dict()
        return Dict("elements" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl)
    else
        elements = vcat(exogen, detdseries, detdstructure, thermal, windsol, transm, cons, inflow, fuel, nuclear)
        startmagdict = JSON.parsefile(joinpath(sti_dataset1, "startmagdict.json"))
        detailedrescopl = JSON.parsefile(joinpath(sti_dataset, "magasin_elspot.json"))
        return Dict("elements" => elements, "elements_ppp" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl)
    end
end



getdataset (generic function with 1 method)

### Run JulES and keep the results

In [10]:

dataset = getdataset(config, weatheryear)
input = JulES.DefaultJulESInput(config, dataset, datayear, weatheryear)
JulES.cleanup_jules(input)
data = JulES.run_serial(input)

Time parameters
  0.000052 seconds (56 allocations: 2.250 KiB)
Handle elements
  0.000833 seconds (14.37 k allocations: 716.344 KiB)
Add local dbs
  0.011991 seconds (720 allocations: 33.359 KiB)
Add local cores
  0.001156 seconds (655 allocations: 25.734 KiB)
Add local input
  2.642034 seconds (368.41 k allocations: 12.495 MiB)
Add local dummyobjects
  0.521665 seconds (1.43 M allocations: 120.557 MiB, 21.19% gc time)
Add local subsystems
Number of shortterm storagesystems 0
Number of longterm storagesystems 19
  0.116675 seconds (1.29 M allocations: 38.739 MiB)
Add local scenmod
  0.011731 seconds (645 allocations: 30.266 KiB)
Add local problem distribution
[(1, 1), (2, 2), (3, 3), (4, 4)]
[(1, 1, 1), (2, 1, 2), (3, 1, 3), (4, 1, 4), (1, 2, 1), (2, 2, 2), (3, 2, 3), (4, 2, 4), (1, 3, 1), (2, 3, 2), (3, 3, 3), (4, 3, 4), (1, 4, 1), (2, 4, 2), (3, 4, 3), (4, 4, 4), (1, 5, 1), (2, 5, 2), (3, 5, 3), (4, 5, 4), (1, 6, 1), (2, 6, 2), (3, 6, 3), (4, 6, 4), (1, 7, 1), (2, 7, 2), (3, 7, 3), (

Row,model,update,solve,other,total
,String,Float64,Float64,Float64,Float64
1,long,0.0600922,0.0613719,0.000148675,0.121613
2,med,0.0764261,0.0446262,0.00024465,0.121297
3,short,0.0223961,0.0610956,0.00206502,0.0855567
4,evp,0.000576142,0.00116488,0.000690603,0.00243163
5,mp,6.78289e-5,0.000386784,0.000930458,0.00138507
6,sp,0.000813418,0.00140462,0.00474206,0.00696009
7,clearing,0.00783346,0.0160617,0.00296988,0.0268651


Row,core,tot,evp_tot,mp_tot,sp_tot,evp_u,evp_s,evp_o,mp_u,mp_s,mp_fin,mp_o,sp_u,sp_s,sp_o
,Any,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,5.0,missing,missing,0.0025246,0.0108905,missing,missing,missing,0.0001236,0.00065625,8.1e-6,0.00173665,0.00153715,0.0016091,0.00774425
2,6.0,missing,missing,0.00266205,0.0123995,missing,missing,missing,0.0001249,0.0007014,7.15e-6,0.0018286,0.0015679,0.00244935,0.0083822
3,7.0,missing,missing,0.0028164,0.0150379,missing,missing,missing,0.0001361,0.00089385,9.6e-6,0.00177685,0.0015756,0.0028401,0.0106221
4,8.0,missing,missing,0.003167,0.0149395,missing,missing,missing,0.00013035,0.0007425,1.07e-5,0.00228345,0.00157295,0.00312455,0.0102421
5,9.0,missing,missing,0.00307135,0.0155729,missing,missing,missing,0.0001448,0.0007177,1.075e-5,0.0021981,0.00157275,0.00269,0.0113101
6,10.0,missing,missing,0.00206815,0.0082357,missing,missing,missing,8.465e-5,0.0003466,5.6e-6,0.0016313,0.0008051,0.0007977,0.0066329
7,1.0,0.0321001,0.0162426,0.0014649,0.0143926,0.0029067,0.0055915,0.00096805,0.00013705,0.0008467,1.53e-5,0.00046585,0.00180425,0.00275835,0.00983
8,2.0,0.0283646,0.0103,0.00293835,0.0151261,0.0028105,0.0055954,0.000236769,0.0001426,0.0009288,8.85e-6,0.0018581,0.0017904,0.00382315,0.0095126
9,3.0,0.0258226,0.009917,0.0029242,0.0129815,0.00266085,0.00552665,0.000216188,0.0001399,0.0007263,9.1e-6,0.0020489,0.0016318,0.003106,0.00824365


Row,subix,tot,evp_tot,mp_tot,sp_tot,evp_u,evp_s,evp_o,mp_u,mp_s,mp_fin,mp_o,sp_u,sp_s,sp_o
,Any,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Any,Any,Any,Any,Float64?,Float64?,Float64?
1,8.0,0.0140684,0.0024171,0.00222595,0.0094253,0.000558,0.00145095,5.10188e-5,7.995e-5,0.00041825,6.7e-6,0.00172105,0.00079945,0.00184575,0.0067801
2,18.0,0.0136875,0.0022332,0.00148245,0.0099719,0.0005552,0.0013579,4.00125e-5,8.015e-5,0.0003964,4.35e-6,0.00100155,0.00092395,0.00169295,0.007355
3,1.0,0.0131388,0.00793795,0.000979,0.0042218,0.0008433,0.00094875,0.000768238,4.96e-5,0.000292,4.05e-6,0.00063335,0.0007543,0.00079285,0.00267465
4,11.0,0.0128664,0.002174,0.0022729,0.0084195,0.00054595,0.00129375,4.17875e-5,8.125e-5,0.0004052,6.6e-6,0.00177985,0.000816,0.0015881,0.0060154
5,7.0,0.0120401,0.00173625,0.00206815,0.0082357,0.00056515,0.0008521,3.9875e-5,8.465e-5,0.0003466,5.6e-6,0.0016313,0.0008051,0.0007977,0.0066329
6,17.0,0.0119028,0.00238295,0.0013958,0.008124,0.0005696,0.00141605,4.96625e-5,7.37e-5,0.00039985,5.9e-6,0.00091635,0.00083175,0.0016485,0.00564375
7,12.0,0.011803,0.001607,0.00180795,0.00838805,0.00056175,0.00070185,4.2925e-5,8.52e-5,0.00033495,4.95e-6,0.00138285,0.0008054,0.00077625,0.0068064
8,14.0,0.0117946,0.0022944,0.0017567,0.0077435,0.0005679,0.00142965,3.71063e-5,7.835e-5,0.00039505,3.65e-6,0.00127965,0.00079925,0.00160425,0.00534
9,16.0,0.0113124,0.0021924,0.00162835,0.0074917,0.00053945,0.001362,3.63687e-5,6.805e-5,0.00045305,6.45e-6,0.0011008,0.00080845,0.0017001,0.00498315


Dict{Any, Any} with 23 entries:
  "demandbalancenames" => SubString{String}["SORLAND", "SORLAND", "SORLAND", "S…
  "endvaluetimes"      => Dict{Any, Any}((1, 12)=>[0.0001766 0.0002924 0.000578…
  "resindex"           => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "areanames"          => ["SORLAND"]
  "priceindex"         => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-2…
  "batmatrix"          => Matrix{Float64}(undef, 128, 0)
  "stateindex"         => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "statematrix"        => [547.673 4078.56 … 0.174767 0.0; 547.673 4078.56 … 0.…
  "statenames"         => ["Reservoir_FINNMARK_hydro_reservoir_max", "Reservoir…
  "resnames"           => ["Reservoir_SORLAND_hydro_reservoir"]
  "demandvalues"       => [1.3 0.0 … 0.0831064 1.40595; 1.3 0.0 … 0.0476586 1.9…
  "batindex"           => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "resmatrix"          => [14.4024; 14.3551; … ; 13.769; 13.753;;]
  "supplybala

### Code to show results
- We don't show any results for this demo

In [ ]:
powerbalancenames = data["areanames"]
prices = data["pricematrix"]
x1 = data["priceindex"]

hydronames = data["resnames"]
hydrolevels = data["resmatrix"]
x2 = data["resindex"]

batterynames = data["batnames"]
batterylevels = data["batmatrix"]
x2 = data["batindex"]

statenames = data["statenames"]
statematrix = data["statematrix"]
x3 = data["stateindex"]

supplyvalues = data["supplyvalues"]
supplynames = data["supplynames"]
supplybalancenames = data["supplybalancenames"]

demandvalues = data["demandvalues"]
demandnames = data["demandnames"]
demandbalancenames = data["demandbalancenames"];

# storagevalues = data["storagevalues"]
# storagenames = data["storagenames"]
# scenarionames = data["scenarionames"]
# shorts = data["shorts"]
# skipfactor = data["skipfactor"];

In [ ]:
# a = 0.5
# b = -4
# c = 10
# # a = 1
# # b = 0
# # c = 5
# # a = 0
# # b = 0
# # c = 1
# numscen = 7
# x = collect(-numscen+1:2:numscen-1)
# y = a .* x .^ 2 .+ x .* b .+ c
# display(y/sum(y)) # show chosen weights

In [ ]:
# Plot prices
idxwohub = findall(x -> !occursin("HUB", x), powerbalancenames) # remove hubs, not active in 2025 dataset
display(plot(x1, prices[:,idxwohub]*100, labels=reshape(powerbalancenames[idxwohub],1,length(powerbalancenames[idxwohub])), size=(800,500), title="Prices", ylabel="€/MWh"))

# # Plot supplies and demands
# maxdemsup = isempty(supplyvalues) ? maximum(demandvalues) : (isempty(demandvalues) ? maximum(supplyvalues) : max(maximum(demandvalues), maximum(supplyvalues)))
# supplychart = plot(x1, supplyvalues,labels=reshape(supplynames,1,length(supplynames)),title="Supply", ylabel = "GWh/h", ylims=(0,maxdemsup))
# demandchart = plot(x1, demandvalues,labels=reshape(demandnames,1,length(demandnames)),title="Demand", ylabel = "GWh/h", ylims=(0,maxdemsup))
sumsupplyvalues = sum(supplyvalues,dims=2)
sumdemandvalues = sum(demandvalues,dims=2)
maxdemsup = isempty(sumsupplyvalues) ? maximum(sumdemandvalues) : (isempty(sumdemandvalues) ? maximum(sumsupplyvalues) : max(maximum(sumdemandvalues), maximum(sumsupplyvalues)))
supplychart = areaplot(x1,sumsupplyvalues,title="Supply", ylabel = "GWh/h")
demandchart = areaplot(x1,sumdemandvalues,title="Demand", ylabel = "GWh/h")
display(plot([supplychart,demandchart]...,layout=(1,2),size=(800,500)))
# display(plot(supplychart,size=(800,500)))

# Plot storages
# display(areaplot(x2, hydrolevels1,labels=reshape(hydronames,1,length(hydronames)),size=(800,500),title="Reservoir levels", ylabel = "TWh")) #
display(areaplot(x2, dropdims(sum(hydrolevels,dims=2),dims=2),labels="Total",size=(800,500),title="Reservoir levels", ylabel = "TWh")) #

# display(areaplot(x1, dropdims(sum(batterylevels,dims=2),dims=2),labels="Total",size=(800,500),title="Short term storage levels", ylabel = "GWh")) #

# Plot list of yearly mean production and demand for each supply/demand TODO: split demand/supply and transmission
meandemand = dropdims(mean(demandvalues,dims=1),dims=1)
meanproduction = dropdims(mean(supplyvalues,dims=1),dims=1)
supplydf = sort(DataFrame(Supplyname = supplynames, Yearly_supply_TWh = meanproduction*8.76),[:Yearly_supply_TWh], rev = true)
demanddf = sort(DataFrame(Demandname = demandnames, Yearly_demand_TWh = meandemand*8.76),[:Yearly_demand_TWh], rev = true)
supplydf[!,:ID] = collect(1:length(supplynames))
demanddf[!,:ID] = collect(1:length(demandnames))
joineddf = select!(outerjoin(supplydf,demanddf;on=:ID),Not(:ID))
pretty_table(joineddf, show_subheader = true)

# Check that total supply equals total demand
pretty_table(combine(joineddf, [:Yearly_supply_TWh, :Yearly_demand_TWh] .=> sum∘skipmissing), show_subheader = true)

# Plot list of yearly income and cost for each supply/demand (only works if exogenprices are collected)
supplyrev = copy(supplyvalues)
for (i,supplybalancename) in enumerate(supplybalancenames)
    idx = findfirst(isequal(supplybalancename), powerbalancenames)
    supplyrev[:,i] .= supplyrev[:,i] .* prices[:,idx]
end
demandrev = copy(demandvalues)
for (i,demandbalancename) in enumerate(demandbalancenames)
    idx = findfirst(isequal(demandbalancename), powerbalancenames)
    demandrev[:,i] .= demandrev[:,i] .* prices[:,idx]
end
meandemandrev = dropdims(mean(demandrev,dims=1),dims=1)
meanproductionrev = dropdims(mean(supplyrev,dims=1),dims=1)
supplyrevdf = sort(DataFrame(Supplyname = supplynames, Yearly_rev_mill€ = meanproductionrev*8.76),[:Yearly_rev_mill€], rev = true)
demandrevdf = sort(DataFrame(Demandname = demandnames, Yearly_cost_mill€ = meandemandrev*8.76),[:Yearly_cost_mill€], rev = true)
supplyrevdf[!,:ID] = collect(1:length(supplynames))
demandrevdf[!,:ID] = collect(1:length(demandnames))
joinedrevdf = select!(outerjoin(supplyrevdf,demandrevdf;on=:ID),Not(:ID))
# pretty_table(joinedrevdf, show_subheader = true)

# Sum revenues and cost
pretty_table(combine(joinedrevdf, [:Yearly_rev_mill€, :Yearly_cost_mill€] .=> sum∘skipmissing), show_subheader = true)

# # Plot storagevalues for each reservoir and scenarios
# maxlongtermstorages = 40
# maxshorttermstorages = 40
# shortindex = x3
# medindex = x3[1:Int(skipfactor):Int(end-skipfactor)]
# numop = length(findall(sn -> occursin("Operative", sn), scenarionames))
# numscen = length(scenarionames) - numop
# j = 0
# k = 0
# for (i, storagename) in enumerate(storagenames)
#     if shorts[i]
#         j += 1
#         j > maxshorttermstorages && continue
#         storagevalues_ = storagevalues[:,:,:]
#         index = shortindex
#     else
#         k += 1
#         k > maxlongtermstorages && continue
#         storagevalues_ = storagevalues[1:Int(skipfactor):Int(end-skipfactor),:,:]
#         index = medindex
#     end
#     p = plot(index, storagevalues_[:,1:numscen,i] * -100, size=(800,500), title="Storagevalues scenario and operative for " * storagename, labels=reshape(scenarionames[1:numscen], 1, numscen), ylabel="€/MWh")
#     plot!(p, index, storagevalues_[:,numscen+1:end,i] * -100, labels=reshape(scenarionames[numscen+1:end], 1, numop), linewidth=5)
#     display(p)
# end